In [1]:
# ============================================================
# 1. Setup (run once)
# ============================================================
import torch
from torch.utils.data import Dataset, DataLoader
import sys, os
sys.path.append(r"C:\Users\Hagai.LAPTOP-QAG9263N\Desktop\Thesis\repositories\ImagiNav")
from modules.autoencoder import AutoEncoder
from modules.unet import UNet
from modules.diffusion import LatentDiffusion
from modules.scheduler import CosineScheduler
from training.diffusion_trainer import DiffusionTrainer



In [2]:

# ============================================================
# 2. Dummy dataset
# ============================================================
class DummyDataset(Dataset):
    def __init__(self, length=32, shape=(3, 64, 64)):
        self.length = length
        self.shape = shape

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        # return tuple to preserve batch dimension
        return (torch.zeros(self.shape),)



train_loader = DataLoader(DummyDataset(length=16), batch_size=4)
val_loader = DataLoader(DummyDataset(length=8), batch_size=4)




In [3]:
# ============================================================
# 3. Instantiate components (aligned latent AE + UNet)
# ============================================================
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define shared latent geometry for AE and UNet
latent_channels = 3
latent_base = 16
image_size = 64

# Autoencoder built via new from_shape API
autoencoder = AutoEncoder.from_shape(
    in_channels=3,
    out_channels=3,
    base_channels=16,
    latent_channels=latent_channels,
    image_size=image_size,
    latent_base=latent_base,
    norm="batch",
    act="relu"
).to(device)

# UNet configured to operate on the same latent space
unet = UNet(
    in_channels=latent_channels,
    out_channels=latent_channels,
    base_channels=16,
    depth=3
).to(device)

scheduler = CosineScheduler(num_steps=10)
latent_diffusion = LatentDiffusion(unet, scheduler, autoencoder)


In [4]:

trainer = DiffusionTrainer(
    unet=unet,
    autoencoder=autoencoder,
    scheduler=scheduler,
    epochs=10,
    log_interval=1,       # log every step
    sample_interval=2,    # create artifacts every 2 steps
    eval_interval=4,
    output_dir="test_outputs",
    ckpt_dir="test_outputs/checkpoints",
)

# ============================================================
# 4. Run one short training cycle
# ============================================================
trainer.fit(train_loader, val_loader)

# ============================================================
# 5. Inspect results
# ============================================================
print("\nTraining complete.")
print("Artifacts saved in:", os.path.abspath(trainer.output_dir))
print("Metric log entries:", len(getattr(trainer, 'metric_log', [])))

# Display one sample of recorded metrics (if available)
if hasattr(trainer, "metric_log") and trainer.metric_log:
    print("Example metrics:", trainer.metric_log[0])


Dataset diversity baseline: 0.0000


Epoch 1/10:   0%|          | 0/4 [00:00<?, ?it/s, loss=1.0782]

Generating...


Epoch 1/10:  25%|██▌       | 1/4 [00:00<00:00,  7.41it/s, loss=1.0776]

Decoding...
[1] loss=1.0782, snr=0.781, cos=-0.035, div=0.002
Generating...


...

Diffusion sampling: 100%|██████████| 10/10 [00:00<00:00, 70.90it/s]


Decoding...
[40] loss=0.9317, snr=0.333, cos=0.306, div=0.002
Generating...


Diffusion sampling: 100%|██████████| 10/10 [00:00<00:00, 53.57it/s]


Decoding...


[Validation @ step 40] loss=0.9771
[Checkpoint] Saved: test_outputs/checkpoints/epoch_10.pt

Training complete.
Artifacts saved in: c:\Users\Hagai.LAPTOP-QAG9263N\Desktop\Thesis\repositories\ImagiNav\notebooks\test_outputs
Metric log entries: 40
Example metrics: {'step': 1, 'loss': 1.0782171487808228, 'snr': 0.7807475328445435, 'cosine': -0.0346669927239418, 'grad_norm': 1.142073154449463, 'diversity': 0.002451847540214658}
